In [1]:
pip install langchain-huggingface

In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [3]:
pip install openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 29.6 MB/s eta 0:00:00


In [4]:
pip install langchain

In [5]:
pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.8 MB/s eta 0:00:00


In [6]:
pip install --upgrade langchain

In [7]:
%pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.1 MB/s eta 0:00:00


In [8]:
!pip install huggingface_hub

In [9]:
%pip install -qU langchain-google-genai

In [10]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key:")


Enter your Google AI API key:··········


In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import PyPDFLoader

def main():
    # Load the PDF
    loader = PyPDFLoader("/content/2309.05519v3.pdf")
    pages = loader.load_and_split()
    print(f"{len(pages)} pages loaded.")

    # Initialize LLM and embeddings
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    # Convert documents to embeddings and store in FAISS
    vector_store = FAISS.from_documents(pages, embeddings)

    # Set up the retriever
    retriever = vector_store.as_retriever()

    # Initialize memory for conversation history
    memory = ConversationBufferMemory(
        memory_key="history", return_messages=True
    )

    # Define prompt template
    prompt_template = ChatPromptTemplate.from_template("""
    Use the following context to answer the user's questions. Keep the conversation history in mind for continuity.

    <context>
    {context}
    </context>

    Conversation History:
    {history}

    User: {input}
    Assistant:
    """)

    # Create document and retrieval chains
    document_chain = create_stuff_documents_chain(llm, prompt_template)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    print("Chatbot is ready! Type 'exit' to end the conversation.")
    print("You can now ask questions based on the provided research papers.")

    # Chat loop
    while True:
        # Get user input
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            print("Conversation ended. Goodbye!")
            break

        # Query the chain
        response = retrieval_chain.invoke({
            "context": (
                "You are an expert summarizing and explaining concepts, methodologies, "
                "and findings from research papers related to Large Language Models (LLMs). "
                "You can provide insights into topics such as model architectures, "
                "training techniques, evaluation metrics, comparisons between models, "
                "and their real-world applications."
            ),
            "input": user_input,
            "history": memory.load_memory_variables({})["history"]
        })

        # Update memory with the conversation
        memory.save_context(
            inputs={"input": user_input},
            outputs={"output": response["answer"]}
        )

        # Display the response
        print(f"Assistant: {response['answer']}")

if __name__ == "__main__":
    main()


48 pages loaded.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-11-5901a1cfb866>:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Chatbot is ready! Type 'exit' to end the conversation.
You can now ask questions based on the provided research papers.
You: What is their methodology that they have followed
Assistant: The methodology followed by NExT-GPT involves three stages:

1.  **Input-side Alignment Learning:** This stage focuses on training a compatible multimodal tokenizer for the frozen LLM.
2.  **Decoding-side Alignment Learning:** This stage involves training the output projection layers using a combination of cross-entropy, l2-distance, and conditional latent denoising loss. Only the parameters of the output projection layers are learnable in this stage.
3.  **End-to-end Instruction-Tuning:** In this final stage, the entire NExT-GPT model is trained using instruction-following datasets. LoRA is used to fine-tune the weights of the LLM, and both the input and output projection layers are trainable.

You: explain about related work based on their work
Assistant: Based on the provided context, here's an expla